# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f25e0298c10>
├── 'a' --> tensor([[-0.6659, -1.4148,  0.1034],
│                   [ 2.3474, -0.5496,  0.6448]])
└── 'x' --> <FastTreeValue 0x7f25e0298400>
    └── 'c' --> tensor([[ 0.7854,  1.0524, -1.2328,  2.0520],
                        [-1.3166, -0.5465,  0.5196, -0.0421],
                        [ 0.3989,  0.0245, -0.4494,  0.1799]])

In [4]:
t.a

tensor([[-0.6659, -1.4148,  0.1034],
        [ 2.3474, -0.5496,  0.6448]])

In [5]:
%timeit t.a

62.1 ns ± 0.0228 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f25e0298c10>
├── 'a' --> tensor([[-0.9354, -1.1591, -1.6061],
│                   [ 0.6637,  0.2719,  0.3867]])
└── 'x' --> <FastTreeValue 0x7f25e0298400>
    └── 'c' --> tensor([[ 0.7854,  1.0524, -1.2328,  2.0520],
                        [-1.3166, -0.5465,  0.5196, -0.0421],
                        [ 0.3989,  0.0245, -0.4494,  0.1799]])

In [7]:
%timeit t.a = new_value

70.7 ns ± 0.0361 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6659, -1.4148,  0.1034],
               [ 2.3474, -0.5496,  0.6448]]),
    x: Batch(
           c: tensor([[ 0.7854,  1.0524, -1.2328,  2.0520],
                      [-1.3166, -0.5465,  0.5196, -0.0421],
                      [ 0.3989,  0.0245, -0.4494,  0.1799]]),
       ),
)

In [10]:
b.a

tensor([[-0.6659, -1.4148,  0.1034],
        [ 2.3474, -0.5496,  0.6448]])

In [11]:
%timeit b.a

58.3 ns ± 0.0423 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.3015,  0.1315,  0.2054],
               [-1.0404, -0.0256, -0.7416]]),
    x: Batch(
           c: tensor([[ 0.7854,  1.0524, -1.2328,  2.0520],
                      [-1.3166, -0.5465,  0.5196, -0.0421],
                      [ 0.3989,  0.0245, -0.4494,  0.1799]]),
       ),
)

In [13]:
%timeit b.a = new_value

495 ns ± 0.0649 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

861 ns ± 0.642 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 16 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 417 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2534f72fd0>
├── 'a' --> tensor([[[-0.6659, -1.4148,  0.1034],
│                    [ 2.3474, -0.5496,  0.6448]],
│           
│                   [[-0.6659, -1.4148,  0.1034],
│                    [ 2.3474, -0.5496,  0.6448]],
│           
│                   [[-0.6659, -1.4148,  0.1034],
│                    [ 2.3474, -0.5496,  0.6448]],
│           
│                   [[-0.6659, -1.4148,  0.1034],
│                    [ 2.3474, -0.5496,  0.6448]],
│           
│                   [[-0.6659, -1.4148,  0.1034],
│                    [ 2.3474, -0.5496,  0.6448]],
│           
│                   [[-0.6659, -1.4148,  0.1034],
│                    [ 2.3474, -0.5496,  0.6448]],
│           
│                   [[-0.6659, -1.4148,  0.1034],
│                    [ 2.3474, -0.5496,  0.6448]],
│           
│                   [[-0.6659, -1.4148,  0.1034],
│                    [ 2.3474, -0.5496,  0.6448]]])
└── 'x' --> <FastTreeValue 0x7f2534f72f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2534f4f580>
├── 'a' --> tensor([[-0.6659, -1.4148,  0.1034],
│                   [ 2.3474, -0.5496,  0.6448],
│                   [-0.6659, -1.4148,  0.1034],
│                   [ 2.3474, -0.5496,  0.6448],
│                   [-0.6659, -1.4148,  0.1034],
│                   [ 2.3474, -0.5496,  0.6448],
│                   [-0.6659, -1.4148,  0.1034],
│                   [ 2.3474, -0.5496,  0.6448],
│                   [-0.6659, -1.4148,  0.1034],
│                   [ 2.3474, -0.5496,  0.6448],
│                   [-0.6659, -1.4148,  0.1034],
│                   [ 2.3474, -0.5496,  0.6448],
│                   [-0.6659, -1.4148,  0.1034],
│                   [ 2.3474, -0.5496,  0.6448],
│                   [-0.6659, -1.4148,  0.1034],
│                   [ 2.3474, -0.5496,  0.6448]])
└── 'x' --> <FastTreeValue 0x7f2534f4fc10>
    └── 'c' --> tensor([[ 0.7854,  1.0524, -1.2328,  2.0520],
                        [-1.3166, -0.5465,  0.5196, -0.0421],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 61.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6659, -1.4148,  0.1034],
                [ 2.3474, -0.5496,  0.6448]],
       
               [[-0.6659, -1.4148,  0.1034],
                [ 2.3474, -0.5496,  0.6448]],
       
               [[-0.6659, -1.4148,  0.1034],
                [ 2.3474, -0.5496,  0.6448]],
       
               [[-0.6659, -1.4148,  0.1034],
                [ 2.3474, -0.5496,  0.6448]],
       
               [[-0.6659, -1.4148,  0.1034],
                [ 2.3474, -0.5496,  0.6448]],
       
               [[-0.6659, -1.4148,  0.1034],
                [ 2.3474, -0.5496,  0.6448]],
       
               [[-0.6659, -1.4148,  0.1034],
                [ 2.3474, -0.5496,  0.6448]],
       
               [[-0.6659, -1.4148,  0.1034],
                [ 2.3474, -0.5496,  0.6448]]]),
    x: Batch(
           c: tensor([[[ 0.7854,  1.0524, -1.2328,  2.0520],
                       [-1.3166, -0.5465,  0.5196, -0.0421],
                       [ 0.3989,  0.0245, -0.4494,  0.1799]],
         

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6659, -1.4148,  0.1034],
               [ 2.3474, -0.5496,  0.6448],
               [-0.6659, -1.4148,  0.1034],
               [ 2.3474, -0.5496,  0.6448],
               [-0.6659, -1.4148,  0.1034],
               [ 2.3474, -0.5496,  0.6448],
               [-0.6659, -1.4148,  0.1034],
               [ 2.3474, -0.5496,  0.6448],
               [-0.6659, -1.4148,  0.1034],
               [ 2.3474, -0.5496,  0.6448],
               [-0.6659, -1.4148,  0.1034],
               [ 2.3474, -0.5496,  0.6448],
               [-0.6659, -1.4148,  0.1034],
               [ 2.3474, -0.5496,  0.6448],
               [-0.6659, -1.4148,  0.1034],
               [ 2.3474, -0.5496,  0.6448]]),
    x: Batch(
           c: tensor([[ 0.7854,  1.0524, -1.2328,  2.0520],
                      [-1.3166, -0.5465,  0.5196, -0.0421],
                      [ 0.3989,  0.0245, -0.4494,  0.1799],
                      [ 0.7854,  1.0524, -1.2328,  2.0520],
                      [-1.3166, -

In [28]:
%timeit Batch.cat(batches)

145 µs ± 434 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

337 µs ± 3.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
